In [32]:
# Generador de letras de The Beatles en base a cadenas de Markov

# Franco Guiragossian
# franco@modeliz.ar

import numpy as np
import string

In [33]:
# Inicializamos diccionarios

inicial = {} # Dict Inicial: Va a generar la primer palabra.
prim_orden = {} # Dict de primer orden: Permite generar la segunda palabra dada la anterior.
seg_orden = {} # Dict de segundo orden: Permite generar palabras dadas las dos palabras anteriores.

In [34]:
# Función para remover puntuaciones

def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [35]:
# Función para agregar una palabras a un diccionario

def agregar_palabra(dicc, pal, idx):
    # Los inputs son el diccionario, la palabra, y valor (índice)
    if pal not in dicc:
        dicc[pal] = []
    dicc[pal].append(idx)

In [36]:
# Populamos los diccionarios en base a las letras, iterando renglón por renglón

for line in open('lyrics.txt'):
    # Tokenizamos: Separamos los renglones palabra por palabra en una lista
    tokens = remove_punctuation(line.rstrip().lower()).split()
    T = len(tokens)
    if T == 1:
        # Si el renglón tiene una sola palabra, saltamos a la próxima iteración:
        continue
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # Si es la primera palabra, la consideramos para el diccionario inicial
            inicial[t] = inicial.get(t, 0.) + 1
            # Defaultea en 0, ya que si una palabra no había sido observada antes, empieza a contarla sumando
            # 1 a 0
        else:
            t_1 = tokens[i-1] # t_1 es la palabra precedente
            if i == T - 1:
                # Si es la última palabra, agrego token especial "FIN" para que
                # el modelo pueda samplear el final de una oración
                agregar_palabra(seg_orden, (t_1, t), 'FIN')
            if i == 1:
                # Si es la segunda palabra, agrego al diccionario de primer orden
                agregar_palabra(prim_orden, t_1, t)
            else:
                # Si no, agrego al diccionario de segundo orden (en base a las dos palabras precedentes)
                t_2 = tokens[i-2]
                agregar_palabra(seg_orden, (t_2, t_1), t)

In [37]:
# Normalizamos las distribuciones
total_inicial = sum(inicial.values())
for t, c in inicial.items():
    inicial[t] = c / total_inicial

In [38]:
# Función para calcular probabilidades de cada palabra en base a las anteriores, en cada diccionario

def calcular_p(ts):
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c / n
    return d

In [39]:
for t_1, ts in prim_orden.items():
    prim_orden[t_1] = calcular_p(ts)

In [78]:
for k, ts in seg_orden.items():
    seg_orden[k] = calcular_p(ts)

In [41]:
# Función para tomar una muestra de una palabra

def samplear_palabra(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t

In [81]:
# Función para generar "n" renglones de texto basado en las letras de The Beatles

def generar_texto(n=4):
    for i in range(n):
        sentence = []
        w0 = samplear_palabra(inicial)
        sentence.append(w0)
        w1 = samplear_palabra(prim_orden[w0])
        sentence.append(w1)
        while True:
            w2 = samplear_palabra(seg_orden[(w0, w1)])
            if w2 == 'FIN':
                # Si sampleo el token especial "FIN", termino el renglón
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [96]:
generar_texto(4)

so come on baby dont care to hear from you
then its far too late
oh believe me darling
gather round all you clowns
